# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml-capstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


***Creating the initial steps and datasets***


In [3]:
import requests
from io import StringIO
import pandas as pd
from azureml.core import Workspace, Dataset

# Replace with your GitHub token and the correct URL
GITHUB_TOKEN = 'removed for privacy purposes'
url = 'https://raw.githubusercontent.com/monaejam/Udacity/main/capston/heart_failure_clinical_records_dataset.csv'

# Fetch the raw CSV content from GitHub
headers = {'Authorization': f'token {GITHUB_TOKEN}'}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = StringIO(response.text)
    df = pd.read_csv(data)
    
    # Print first few rows to confirm data retrieval
    print(df.head())
else:
    print(f"Failed to retrieve file. Status code: {response.status_code}")

# Now, you can proceed to register this dataset in your Azure ML Workspace

# Connect to your workspace
ws = Workspace.from_config()  # Or provide parameters manually if needed

# Register the dataset in Azure ML Workspace
dataset = Dataset.Tabular.register_pandas_dataframe(df, target=ws.get_default_datastore(), name="heart_failure_dataset")

# Convert to pandas DataFrame and describe the dataset
df = dataset.to_pandas_dataframe()
df.describe()


    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2       

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


**AUTO ML STEPS**

In [28]:
import logging
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Experiment, Dataset
import os

# Check and display the current working directory
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# Define the project folder path (ensure this is a valid path)
project_folder = os.path.join(current_directory, 'automl_classification')

# Check if the project folder exists
if not os.path.exists(project_folder):
    os.makedirs(project_folder)
    print(f"Created project folder at: {project_folder}")
else:
    print(f"Project folder already exists at: {project_folder}")

# Connect to your Azure ML workspace
ws = Workspace.from_config()

# URL of the dataset
url = 'https://raw.githubusercontent.com/monaejam/Udacity/main/capston/heart_failure_clinical_records_dataset.csv'

# Check if the dataset is already registered in the workspace
key = "heart_failure_dataset"
if key in ws.datasets.keys():
    dataset = ws.datasets[key]  # Load the dataset if it exists
else:
    # Create AML Dataset from the URL and register it into Workspace
    dataset = Dataset.Tabular.from_delimited_files(url)
    dataset = dataset.register(workspace=ws, name=key, description="Heart failure dataset for Udacity")

# Define the AutoML settings
automl_settings = {
    "n_cross_validations": 3,  # Cross-validation
    "primary_metric": 'AUC_weighted',  # Use AUC_weighted as the primary metric
    "enable_early_stopping": True,
    "max_concurrent_iterations": 6,  # Number of models to train concurrently
    "experiment_timeout_minutes": 30,  # 30-minute timeout for the experiment
    "verbosity": logging.INFO,
    "enable_ensembling": True,  # Enable ensembling
    "enable_stack_ensemble": True  # Enable stacked ensembling
}
automl_config = AutoMLConfig(compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            blocked_models = ['XGBoostClassifier'],
                            **automl_settings
                            )
experiment_name = 'automl-experiment'
automl_experiment = Experiment(ws, experiment_name)
# Submit the AutoML experiment
try:
    run = automl_experiment.submit(config=automl_config)
    run.wait_for_completion(show_output=True)
    run_details = run.get_details()
    print(run_details)
except Exception as e:
    print(f"Failed to submit AutoML run. Error: {e}")



Current Working Directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/monaejam2/code/Users/monaejam/capston
Project folder already exists at: /mnt/batch/tasks/shared/LS_root/mounts/clusters/monaejam2/code/Users/monaejam/capston/automl_classification
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl-experiment,AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
from azureml.widgets import RunDetails
from azureml.core import Experiment
from azureml.train.automl import AutoMLConfig
# Use the RunDetails widget to show real-time progress
RunDetails(run).show()
for children_run in run.get_children():
    print('------')
    print(children_run)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_49,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_50,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_48,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_47,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_46,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_45,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: automl-experiment,
Id: AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_44,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: automl-experimen

In [42]:
# Retrieve the best run and the best model
best_run, fitted_model = run.get_output()

# Display the best run and model details
print("Best Run ID: ", best_run.id)

# List all the metrics for the best run
best_run_metrics = best_run.get_metrics()
print("\nBest Run Metrics:")
for metric_name, metric_value in best_run_metrics.items():
    print(f"{metric_name}: {metric_value}")

# List all properties of the best run
best_run_properties = best_run.get_properties()
print("\nBest Run Properties:")
for property_name, property_value in best_run_properties.items():
    print(f"{property_name}: {property_value}")

# Print details of the best model (this will vary based on the model)
print("\nFitted Model Details:")
print(fitted_model)

Best Run ID:  AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_49

Best Run Metrics:
f1_score_macro: 0.8023919673695813
f1_score_micro: 0.8293265993265994
f1_score_weighted: 0.828183807707641
AUC_micro: 0.913528446076931
precision_score_macro: 0.8104504133915897
precision_score_micro: 0.8293265993265994
recall_score_micro: 0.8293265993265994
weighted_accuracy: 0.8482837841144656
average_precision_score_micro: 0.9127800571924563
log_loss: 0.41996110374304546
matthews_correlation: 0.6153414404364136
precision_score_weighted: 0.8367121531738287
recall_score_macro: 0.806032140514899
average_precision_score_weighted: 0.912954000378866
average_precision_score_macro: 0.8903363057861529
accuracy: 0.8293265993265994
norm_macro_recall: 0.6120642810297983
AUC_weighted: 0.9075917789710894
AUC_macro: 0.9075917789710893
recall_score_weighted: 0.8293265993265994
balanced_accuracy: 0.806032140514899
confusion_matrix: aml://artifactId/ExperimentRun/dcid.AutoML_a297385f-e85e-47e0-bcec-fee9f6a04007_49/confusi

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [38]:
model=best_run.register_model(model_name='best_run_capston_automl', model_path='outputs/model.pkl')
model

Model(workspace=Workspace.create(name='Udacity_1', subscription_id='d2d90bd8-e567-4097-88c9-9532cc375686', resource_group='cloud_shell'), name=best_run_capston_automl, id=best_run_capston_automl:3, version=3, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:



inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [67]:
%run endpoint.py


{'Results': [1, 1]}
++++++++++++++++++++++++++++++
Expected result: [true, true], where 'true' means '1' as result in the 'DEATH_EVENT' column


TODO: In the cell below, print the logs of the web service and delete the service

In [59]:
print(service.get_logs())


2024-09-15T22:09:28,945770546+00:00 - rsyslog/run 
2024-09-15T22:09:28,949643034+00:00 - gunicorn/run 
2024-09-15T22:09:28,950665100+00:00 | gunicorn/run | 
2024-09-15T22:09:28,954019372+00:00 | gunicorn/run | ###############################################
2024-09-15T22:09:28,958229740+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-09-15T22:09:28,960566496+00:00 | gunicorn/run | ###############################################
2024-09-15T22:09:28,962627889+00:00 | gunicorn/run | 
2024-09-15T22:09:28,966444974+00:00 - nginx/run 
2024-09-15T22:09:28,969409561+00:00 | gunicorn/run | 
2024-09-15T22:09:28,976272654+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240712.v5
2024-09-15T22:09:28,978827148+00:00 | gunicorn/run | 
2024-09-15T22:09:28,982545659+00:00 | gunicorn/run | 
2024-09-15T22:09:28,983678639+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.


In [60]:
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')


In [62]:
print(fitted_model)


Pipeline(steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/monaejam2/code/Users/monaejam/capston')),
                ('prefittedsoftvotingclassifier',
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('27', Pipeline(s..., RandomForestClassifier(class_weight='balanced', max_features=0.2, min_samples_leaf=0.035789473684210524, min_samples_split=0.056842105263157895, n_estimators=10, n_jobs=1))]))], flatten_transform=False, weights=[0.35714285714285715, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142]))])
